# [Kubernetes](https://kubernetes.io/pt/)

Kubernetes (K8s) é um produto Open Source utilizado para automatizar a implantação, o dimensionamento e o gerenciamento de aplicativos em contêiner.

## Overview


**Master**  
Gerencia a alocação dos recursos no cluster.

[**Nodes**](https://kubernetes.io/docs/concepts/architecture/nodes/)  
Executa os contêineres gerenciados pelo Cluster K8s.   
O Kubernetes executa sua carga de trabalho colocando contêineres em pods para serem executados em nós(Node). 

[**Namespace (ns)**](https://kubernetes.io/docs/concepts/overview/working-with-objects/namespaces/)  
São clusters virtuais. Os namespaces são uma forma de dividir os recursos do cluster.
Alguns exemplos de utilização: ambiente (DEV, HOM, PROD), produto, aplicação, time, ...
Os recursos do kubernetes são criados no namespace kube-system.  

[**Deployment (deploy)**](https://kubernetes.io/docs/concepts/workloads/controllers/deployment/)  
Descreve o estado desejado do POD. 

[**ReplicaSet**](https://kubernetes.io/docs/concepts/workloads/controllers/replicaset/)  
Recurso responsável por monitorar e manter os PODs em execução. Ele garante que o número específico de réplicas de PODs sejam executadas ao mesmo tempo.
Caso um POD seja deletado, o ReplicaSet irá criar um novo de acordo com o estado desejado.

[**POD**](https://kubernetes.io/docs/concepts/workloads/pods/)  
Representa um conjunto de contêiners em execução no Cluster.

[**Services (svc)**](https://kubernetes.io/docs/concepts/services-networking/service/)  
Uma maneira abstrata de expor uma aplicação em execução em um conjunto de Pods como um serviço de rede.  
Tipos de Serviço:
* **ClusterIP** (Padrão): Expõe o serviço em um IP interno do cluster. A escolha desse valor torna o serviço acessível apenas de dentro do cluster.
* **NodePort**: Cria uma porta estática exponto o serviço pelos nodes. O serviço fica acessível via \<Node IP>:\<Node Port>
* **LoadBalancer**: Expõe o serviço externamente usando o balanceador de carga de um **provedor de nuvem**. 
* **ExternalName**: Mapeia o serviço para o conteúdo do campo externalName (por exemplo foo.bar.example.com).

Você também pode usar o Ingress para expor seu serviço. Ingress não é um tipo de serviço, mas atua como o ponto de entrada para seu cluster. Ele permite que você consolide suas regras de roteamento em um único recurso, pois pode expor vários serviços no mesmo endereço IP.   


## Preparando o Acesso ao Ambiente    

In [ ]:
export KUBECONFIG=$PWD/kubeconfig

## Consultando nosso Cluster

1. Consultando a Versão

In [ ]:
kubectl version

2. Consultando as informações do Cluster

In [ ]:
kubectl cluster-info

3. Listando os nós do cluster

In [ ]:
kubectl get nodes

4. Verificando detalhes do node

    Informe o nome do node exibido no passo anterior e execute o comando.

In [ ]:
kubectl describe node <insert-node-name-here>

5. Verificando Processamento do Node

In [ ]:
kubectl top nodes

## Namespaces

1. Consultando o namespace (ns)

In [ ]:
kubectl get namespace

2. Criando namespace (ns)

    Escolha um nome e crie seu namespace

In [ ]:
myNamespace=""
kubectl create ns $myNamespace

3. Verificando os detalhes do namespace

In [ ]:
kubectl describe ns $myNamespace

## Subindo uma Aplicação

Versionamos os recursos (namespace, deploy, service, ...) do cluster em arquivos YAML e para publicar estes recursos no Cluster k8s, utilizamos o comando:

```
kubectl apply -f ./my-resource.yaml 
```

### Deployment

    1. Criando deployment

In [ ]:
cat <<EOF | kubectl -n $myNamespace apply -f -
apiVersion: apps/v1
kind: Deployment
metadata:
  name: helloworld
spec:
  replicas: 1
  selector:
    matchLabels:
      app: helloworld
  template:
    metadata:
      labels:
        app: helloworld
    spec:
      containers:
      - name: helloworld
        image: docker.io/istio/examples-helloworld-v2
        resources:
          requests:
            cpu: "100m"
            memory: "64Mi"
          limits:
            memory: "128Mi"
            cpu: "250m"            
        imagePullPolicy: IfNotPresent #Always
        ports:
        - containerPort: 5000
EOF        

    O deployment também pode ser criado com o códgio abaixo

```
kubectl -n $myNamespace create deployment helloworld --image=docker.io/istio/examples-helloworld-v2
```

2. Consultando o deployment (deploy)

In [ ]:
kubectl -n $myNamespace get deploy helloworld 

3. Consultando o POD

In [ ]:
kubectl -n $myNamespace get pod #Para mais detalhes, trocar o comando get por describe

4. Verificando o log do POD

    Informe o nome do POD apresentado no comando anterior para executar os passos  4, 5 e 6.

In [ ]:
kubectl -n $myNamespace logs <nome-do-POD>

5. Conectando no POD

In [ ]:
kubectl -n $myNamespace exec -it <nome-do-POD> bash

6. Deletando o POD

In [ ]:
kubectl -n $myNamespace delete pod <nome-do-POD>

    Podemos verificar que o POD será recriado.
    O ReplicaSet garante que o aplicação mantanha o estado desejado configurado no deployment.

In [ ]:
kubectl -n $myNamespace get pod 

### Services

1. Criando o serviço

In [ ]:
cat <<EOF | kubectl -n $myNamespace apply -f -
apiVersion: v1
kind: Service
metadata:
  name: helloworld
  labels:
    app: helloworld
spec:
  ports:
  - port: 8080       #Porta para acesso ao Serviço
#   nodePort: 30800  #Porta para acesso aos nós do Cluster (range default: 30000-32767)
    name: http
    targetPort: 5000 #Porta para acesso ao POD   
  selector:
    app: helloworld
  type: LoadBalancer    
EOF    

2. Consultando o Serviço (svc)

In [ ]:
kubectl -n $myNamespace get svc 

    Para acessar o serviço, verifique o IP Externo liberado e acesse conforme abaixo  
* [http://\<ip_externo_loadbalance>:8080/hello](http://ip_externo_loadbalance:8080/hello)

3. Excluindo os Recursos

In [ ]:
kubectl -n $myNamespace delete svc helloworld
kubectl -n $myNamespace delete deploy helloworld
kubectl -n $myNamespace delete ns $myNamespace

> Obs.: Deletando o namespace, todos os recursos gerenciados por ele também são excluídos

[<img align="left" src="media/voltar.png" width="100"/>](01_Preparacao.ipynb)[<img align="right" src="media/avancar.png" width="100"/>](03_Helm.ipynb)
